In [51]:
from langchain_core.documents import Document


docs = [
    Document(page_content="LangChain helps build LLM applications using modular components"),
    Document(page_content="LangChain provides tools for building applications powered by large language models"),
    Document(page_content="You can build LLM-based applications using LangChain and vector databases"),
    
    Document(page_content="Pinecone is a managed vector database for similarity search"),
    Document(page_content="Vector databases like Pinecone and Chroma are used for semantic search"),
    Document(page_content="Chroma is an open-source vector store for embeddings"),
    
    Document(page_content="Hybrid search combines dense embeddings with sparse keyword matching"),
    Document(page_content="Dense retrieval captures semantic meaning using vector embeddings"),
    Document(page_content="Sparse retrieval uses TF-IDF or BM25 for exact keyword search"),
    
    Document(page_content="BM25 is a ranking function widely used in information retrieval"),
    Document(page_content="TF-IDF converts text into sparse vectors based on term frequency"),
    
    Document(page_content="The Eiffel Tower is located in Paris, France"),
    Document(page_content="Paris is the capital city of France")
]


In [52]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
embeddings=HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"

)

In [53]:
from langchain_community.vectorstores import Chroma 
vectorstore=Chroma.from_documents(
    documents=docs,
    embedding=embeddings
)

In [54]:


query="build LLM applications using vector databases"
dense_results=vectorstore.similarity_search_with_score(query,k=5)

print("Dense Results:\n")

for doc,distance in dense_results:
    print(f"{distance:.4f}-->{doc.page_content}")

Dense Results:

0.4040-->You can build LLM-based applications using LangChain and vector databases
0.4040-->You can build LLM-based applications using LangChain and vector databases
0.4040-->You can build LLM-based applications using LangChain and vector databases
0.4040-->You can build LLM-based applications using LangChain and vector databases
0.9638-->Vector databases like Pinecone and Chroma are used for semantic search


In [55]:
from langchain_community.retrievers import BM25Retriever

bm25=BM25Retriever.from_documents(docs)
bm25.k=5

sparse_result=bm25.invoke(query)

print("\nSparse Result:\n")
for doc in sparse_result:
    print(doc.page_content)


Sparse Result:

LangChain helps build LLM applications using modular components
You can build LLM-based applications using LangChain and vector databases
Dense retrieval captures semantic meaning using vector embeddings
Vector databases like Pinecone and Chroma are used for semantic search
LangChain provides tools for building applications powered by large language models


In [56]:
alpha = 0.5  # weight for dense retrieval

scores = {}

# Dense scores
for doc, distance in dense_results:
    dense_score = 1 / (1 + distance)  # convert distance → similarity
    scores[doc.page_content] = alpha * dense_score

# Sparse scores (rank-based)
for rank, doc in enumerate(sparse_result):
    sparse_score = 1 / (rank + 1)
    scores[doc.page_content] = scores.get(doc.page_content, 0) + (1 - alpha) * sparse_score


In [57]:
hybrid_results=sorted(
    scores.items(),
    key=lambda x:x[1],
    
    reverse=True
)

print("\nFinal Hybrid Ranking:\n")
for text, score in hybrid_results:
    print(f"{score:.3f} → {text}")


Final Hybrid Ranking:

0.606 → You can build LLM-based applications using LangChain and vector databases
0.500 → LangChain helps build LLM applications using modular components
0.380 → Vector databases like Pinecone and Chroma are used for semantic search
0.167 → Dense retrieval captures semantic meaning using vector embeddings
0.100 → LangChain provides tools for building applications powered by large language models


In [58]:
top_k = 3

retrieved_context = "\n".join(
    [text for text, _ in hybrid_results[:top_k]]
)

retrieved_context


'You can build LLM-based applications using LangChain and vector databases\nLangChain helps build LLM applications using modular components\nVector databases like Pinecone and Chroma are used for semantic search'

In [59]:
from langchain_core.prompts import PromptTemplate

rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an intelligent assistant.

Use the following retrieved context to answer the question.
If the answer is not present in the context, say "I don't know".

Context:
{context}

Question:
{question}

Answer:
"""
)


In [60]:
final_prompt = rag_prompt.format(
    context=retrieved_context,
    question=query
)

print(final_prompt)



You are an intelligent assistant.

Use the following retrieved context to answer the question.
If the answer is not present in the context, say "I don't know".

Context:
You can build LLM-based applications using LangChain and vector databases
LangChain helps build LLM applications using modular components
Vector databases like Pinecone and Chroma are used for semantic search

Question:
build LLM applications using vector databases

Answer:



In [63]:
from transformers import pipeline

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_new_tokens=200
)


Device set to use cpu


In [64]:
response = llm(final_prompt)

print("Answer:\n")
print(response[0]["generated_text"])


Answer:

I don't know
